# 卷积自动编码器在手写数字去噪中的应用研究


本 Notebook 展示了基于 **卷积自动编码器（Convolutional Autoencoder, CAE）** 进行手写数字去噪的研究过程。数据集使用 **MNIST** 手写数字数据集，并添加高斯噪声模拟噪声数据。主要步骤包括：

1. 数据预处理与噪声添加
2. 卷积自动编码器模型构建
3. 模型训练与验证
4. 去噪效果可视化与评估


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# 1. 加载 MNIST 数据集
(x_train, _), (x_test, _) = mnist.load_data()

# 数据归一化
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# 数据维度调整 (N, 28, 28, 1)
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

# 2. 添加高斯噪声
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# 限制噪声范围在 [0, 1]
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# 数据可视化 (原始数据 vs 噪声数据)
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
    plt.title("Original")
    plt.axis('off')
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28), cmap='gray')
    plt.title("Noisy")
    plt.axis('off')
plt.show()


## 构建卷积自动编码器模型

In [ ]:

# 3. 卷积自动编码器模型
input_img = Input(shape=(28, 28, 1))

# 编码器
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# 解码器
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# 定义模型
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# 模型结构展示
autoencoder.summary()


## 模型训练

In [ ]:

# 4. 模型训练
autoencoder.fit(x_train_noisy, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

# 保存模型
autoencoder.save("denoising_autoencoder.h5")


## 去噪效果可视化

In [ ]:

# 5. 去噪结果展示
decoded_imgs = autoencoder.predict(x_test_noisy)

n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    # 噪声图像
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test_noisy[i].reshape(28, 28), cmap='gray')
    plt.title("Noisy")
    plt.axis('off')
    
    # 去噪后的图像
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
    plt.title("Denoised")
    plt.axis('off')
plt.show()


## 总结与分析


本实验通过卷积自动编码器成功去除了 MNIST 数据集中添加的高斯噪声，验证了自动编码器在图像去噪任务中的有效性。

- **模型结构**：采用卷积层与池化层实现特征提取和重建。
- **去噪效果**：通过对比噪声图像与重建图像，直观展示了去噪效果。
- **实验结果**：去噪后的图像清晰度显著提升，验证了模型的实用性。

进一步优化可考虑增加模型深度、调整训练参数或使用其他变体如 **变分自动编码器（VAE）**。
